In [1]:
from sentence_transformers import SentenceTransformer
from PIL import Image
import os

model = SentenceTransformer("clip-ViT-B-32")
model = model.to('cuda')


/home/adam/miniconda3/envs/cells/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
encodings = []
classes = []
path = []
for image_class in os.listdir('cells_final'):
    for img in os.listdir(f'cells_final/{image_class}'):
        path.append(f'cells_final/{image_class}/{img}')
        encodings.append(model.encode(Image.open(f'cells_final/{image_class}/{img}')))
        classes.append(image_class)
                         
        

In [3]:
import pandas as pd
df = pd.DataFrame()
df['embeddings'] = encodings
df['classes'] = classes
df['path'] = path
#df.to_csv('embedings.csv')

In [4]:
df['class'] = classes

In [6]:
from sklearn.decomposition import PCA
import numpy as np
# Perform PCA
pca = PCA(n_components=2)
pca_embs = pca.fit_transform(np.array(encodings))

# Add to dataframe for convenience
df['pca_x'] = pca_embs[:, 0]
df['pca_y'] = pca_embs[:, 1]


In [ ]:
import umap.umap_ as umap
import numpy as np

# Perform UMAP
umap_reducer = umap.UMAP(n_components=2, metric="euclidean", n_jobs=5, min_dist=0.8, n_neighbors=500)
umap_embs = umap_reducer.fit_transform(np.array(encodings))

# Add to dataframe for convenience
df['umap_x'] = umap_embs[:, 0]
df['umap_y'] = umap_embs[:, 1]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define figure size and number of subplots
FS = (18, 6)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=FS)

# t-SNE plot


# PCA plot
sns.scatterplot(ax=axes[0], x='pca_x', y='pca_y', hue='class', data=df, palette='tab10', alpha=0.1)
axes[0].set_title('PCA')
axes[0].set_xlabel('pca_x')
axes[0].set_ylabel('pca_y')

# UMAP plot
sns.scatterplot(ax=axes[1], x='umap_x', y='umap_y', hue='class', data=df, palette='tab10', alpha=0.1)
axes[1].set_title('UMAP')
axes[1].set_xlabel('umap_x')
axes[1].set_ylabel('umap_y')

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import numpy as np
np.save('array.npy', np.array(encodings))

In [8]:
df.to_csv('reduced.csv')